https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery

In [1]:
import httplib2
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import random
import re
import datetime
from datetime import date, timedelta, datetime
#GTM API Connection
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
#BigQuery API Connection
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# TODO(developer): Set key_path to the path to the service account key file.
key_path = "traynak_service_account.json"
scope = ["https://www.googleapis.com/auth/cloud-platform"]
credentials = service_account.Credentials.from_service_account_file(key_path,scopes=scope,)

client = bigquery.Client(credentials=credentials,project=credentials.project_id)

In [3]:
# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

global_air_quality


In [4]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

In [5]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [6]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,location
0,"BTM Layout, Bengaluru - KSPCB"
1,"BTM Layout, Bengaluru - KSPCB"
2,"BTM Layout, Bengaluru - KSPCB"
3,"BTM Layout, Bengaluru - KSPCB"
4,"BTM Layout, Bengaluru - KSPCB"


In [7]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()
us_cities.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Aberdeen,BROWN,US,no2,14.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,1.0
1,Aberdeen,BROWN,US,o3,65.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,8.0
2,Aberdeen,BROWN,US,pm10,10.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,24.0
3,Aberdeen,BROWN,US,pm25,5.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,24.0
4,Freeport South Avenue I C1012,Houston,US,so2,0.0001,2016-03-06 18:00:00+00:00,ppm,Texas,28.964430,-95.354830,1.0


In [8]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
New York-Northern New Jersey-Long Island    60
Riverside-San Bernardino-Ontario            60
Name: city, dtype: int64

In [9]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = "job" 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 421094712 bytes.


In [10]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/traynak-78114548/queries/8c25ab80-43bb-4e83-a585-3d293b14b752?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000. 421527552 or higher required.

(job ID: 8c25ab80-43bb-4e83-a585-3d293b14b752)

             -----Query Job SQL Follows-----             

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT score, title
   3:        FROM `bigquery-public-data.hacker_news.full`
   4:        WHERE type = "job" 
   5:        
    |    .    |    .    |    .    |    .    |    .    |

In [11]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

1.8835261268448344